# SRH Hospital Appointment Booking System

I used the name 'SRH' because I did not want to use any other name due to copyright issues. I am a student at SRH university so taking the leverage of using the university's name :)

## Please not I have used patient numbers range from T1000 - T2499 so when entering patient number please use valid number within the range :)

In [73]:
# Installation to ensures required library is/are available

!pip install ipywidgets --quiet

# Section 1: Libraries and Widgets

    - Importing necessary Python libraries and setting up widgets
    - Required for building the interactive interface


In [63]:
import ipywidgets as widgets
import datetime
from ipywidgets import Button, Output
from IPython.display import display
from datetime import timedelta
#patient number input widget
patient_number_input = widgets.Text(
    placeholder='Enter patient number',
    style={'description_width': 'initial'}
)

# Section 2: Initiation

    - Definining range for patient numbers to be used in the input widget.

### Challenges

    - Capital 'T' which appears in front of the patient number (different hospitals use this letter as an indication of hospital number, trust number, or clinic number — in this case, it refers to trust number as a lot of hospitals work under trusts or operate as a trust). So, I had to make it not a case-sensitive letter in case the patient enters the right number but doesn't capitalise 't', it should still work and not show an invalid message. (section 7 uses .upper()). The idea of introducing patient number into booking system was due to patient confidentiality.

In [64]:
#patient number range (more than 2000 appointments are available per year but I have added 1500 considering sicknesses or cancellations)
valid_patient_numbers = [f'T{str(i).zfill(3)}' for i in range(1000, 2500)]

suppress_speciality_observer = False

# Section 3: Buttons - Booking & Confirmation

    - Defining and styling buttons for booking appointments and confirmation (confirm/cancel).

In [65]:
#setup book button
book_button= Button(description = 'Book')
book_button.style.button_color = 'grey'

# creating output for patient appointment status message
output_patient_status = Output()

# definining input section
input_section = widgets.VBox([widgets.HBox([patient_number_input, book_button]), output_patient_status])

#setup confirm button
confirm_button = widgets.Button(description = 'Confirm')
confirm_button.style.button_color = 'grey'

#setup cancel button
cancel_button = widgets.Button(description = 'Cancel')
cancel_button.style.button_color = 'grey'

#display confirmation buttons horizontally 
confirmation_buttons = widgets.HBox([confirm_button, cancel_button])

# Section 4: Dropdowns - Speciality, Date & Time

    - Setting up dropdown widgets to allow users to select medical speciality, appointment date, and time slot.

### Challenges

    - The speciality dropdown had a problem, it would select the first speciality by default. So, the program wouldn't trigger since the first option was not 'new' selection. I had to include a blank/empty dropdown as the first option, so the user is forced to make a selection every time.
    
    - I used a dropdown instead of a calendar because I could not limit the calendar to weekdays and the current year. The dropdown only shows weekdays and the current year. Similary, I used dropdown for time to limit the time to 9 to 5.

    - Defining which specialities to include was initially a challenge. Eventually, I added 5 example specialities—these are arbitrary and can be customised as needed by healthcare provider. This is not a technical challenge, but a design note.

In [66]:
# add specialities picker dropdown
speciality_selection = widgets.Dropdown(
    options = ['','Surgical', 'Endocrinology', 'Neurology', 'Dental', 'Oncology'])

# add date picker dropdown
date_selection = widgets.Dropdown(options = [], description = 'Select a date')

# add time picker dropdown
time_selection = widgets.Dropdown(options= [], description = 'Select a time')

# Section 5: Output Placeholders for Dynamic Display

    - Setting up outputs to dynamically display messages, instructions, and confirmation prompts.

### Challenges

    - Even though the booking process is intuitive, I added instructions with each input to guide the user step-by-step. For example, after entering the patient number, user is instructed to select a speciality through a printed message.

In [67]:
#outputs placeholders
output_speciality= Output()
output_date = Output()
output_time = Output()
output_confirmation = Output()
output_confirm = Output()
output_cancel= Output()

# Section 6: Date and Time Setup

    - Defining working days, clinic hours, and mappings between specialities and days for scheduling - working schedule limited to weekdays

In [68]:
# add current year
current_year = datetime.datetime.now().year
start_date = datetime.date(current_year, 1, 1) # defining start date as January first where month comes first in the argument
end_date = datetime.date(current_year, 12, 31) # defining end date with month first and date after
day_one = timedelta(days=1)

# add working days
weekdays = []
while start_date <= end_date:
    if start_date.weekday() < 5:
        weekdays.append(start_date)       
    start_date += day_one # move forward by one day

# add working hours
clinic_hours = list(range(9, 17))
booked = {} #dictionary to store booked appointments
booked_slots = {} # dictionary to store booked time slots

# add corresponding days to each specaility
corresponding_days_to_speciality = {
    0: 'Surgical',                   #Monday
    1: 'Endocrinology',              #Tuesday
    2: 'Neurology',                  #Wednesday
    3: 'Dental',                     #Thursday
    4: 'Oncology'                    #Friday
}

# Section 7: Button Click and Option Selection Functions

    - Functions to operate when buttons (book, confirm, cancel) clicked and dropdowns (specialities, date, and time) selected - including booking logic, filtering available slots, and displaying options to the user.

### Challenges

    - When I booked the first patient and tried to book a second one, the speciality dropdown remained on the previous selection. For example, if the first appointment was booked in "Surgical", trying to book the second one in 'Surgical' again wouldn’t trigger the date function. But if I picked another speciality like 'Oncology', it worked. To solve this, I implemented a reset function for the speciality selection each time a new patient starts a booking.

    - Just like resetting the speciality, I had to reset both the date and time fields. Otherwise, previously selected values would persist, causing logic issues. The patient number field also needed to be reset after a successful booking. Without clearing it, the old number stayed visible. However, I added the patient number to the confirmation message so even after clearing the input, the patient can still see their appointment confirmation details along with patient number.

    - I was able to book the same appointment for the same patient multiple times. To prevent this, I added a condition that checks if a patient already has an appointment and displays a message if they do. I was also able to book the same time and date for multiple patients even if it was already taken. To avoid this, I filtered out already booked slots so that they are not available in the dropdown again.

In [69]:
#defining a function when book button is clicked
def on_book_click(book):
    global suppress_speciality_observer
    patient_number = patient_number_input.value.strip().upper()
    output_patient_status.clear_output()
    output_confirm.clear_output()
    with output_patient_status:
        if patient_number in booked:
            existing_date, existing_time = booked[patient_number]
            print(f'You already have an appointment on {existing_date} at {existing_time}:00.')
        elif patient_number in valid_patient_numbers:
            clear_all()
            suppress_speciality_observer = True # stop auto trigger of the function
            speciality_selection.value = None #resetting specaility dropdown to empty slot
            suppress_speciality_observer = False # allow triggering after resetting
            with output_speciality:
                print('') # to add space for clear visuals
                print('Please select the speciality:')
                display(speciality_selection)
        else:
            print("Invalid patient number. Please try again.")

# defining a function when speciality is picked
def on_pick_a_speciality(change):
# define date selection value
    if suppress_speciality_observer:
        return
    picked_a_speciality = change.new
    output_date.clear_output()

    if not picked_a_speciality:
        return
    
    #Filter available days for the corresponding speciality
    matching_days = [
        days for days in weekdays
        if corresponding_days_to_speciality[days.weekday()] == picked_a_speciality
        and len(booked_slots.get(days,[])) < len(clinic_hours)
    ]
    
    #Formatting date
    date_format =[(days.strftime("%A, %b %d %Y"), days) for days in matching_days]
    date_selection.options = date_format
   
    #displaying date dropdown
    with output_date:
        print('') # to add space for clear visuals
        print('Please select your desired appointment date:')
        display(date_selection)  

#defining a function when a date is picked
def on_pick_a_date(change):
    picked_a_date = date_selection.value
    if picked_a_date:
        booked_hours = booked_slots.get(picked_a_date, [])
        available_hours = [hour for hour in clinic_hours if hour not in booked_hours]
        formatted_hours = [(f"{hour:02d}:00", hour) for hour in available_hours]
        time_selection.options = formatted_hours
        time_selection.value = None # resetting time selection
        output_time.clear_output()
        output_confirmation.clear_output()
        #displaying time dropdown
        with output_time:
            print('') # to add space for clear visuals
            print('Please select your desired appointment time:')
            display(time_selection)

#defining a function when a time is picked
def on_pick_a_time(change):
    picked_a_time = change.new
    if picked_a_time:
        output_confirmation.clear_output()
        #displaying confirmation buttons
        with output_confirmation:
            print('') # to add space for clear visuals
            print('Please confirm your appointment')
            display(confirmation_buttons)

#defining a function when confirm button is clicked
def on_confirm_click(confirm):
# define date selection value
    reserved_date = date_selection.value
    reserved_time = time_selection.value
    patient_number = patient_number_input.value.strip().upper()
    if reserved_date and reserved_time is not None:
        output_confirm.clear_output()
        if patient_number in booked:
            existing_date, existing_time = booked[patient_number]
            with output_confirm:
                print(f'You already have an appointment on {existing_date} at {existing_time}:00.')
        else:
            booked[patient_number] = (reserved_date, reserved_time)
            if reserved_date in booked_slots:
                booked_slots[reserved_date].append(reserved_time)
            else:
                booked_slots[reserved_date] = [reserved_time]

            clear_all()
            #Clear the patient number input after successful booking
            patient_number_input.value = ''
            with output_confirm:
                print(f'Appointment successfully booked for {patient_number} on {reserved_date} at {reserved_time}:00.')

#defining a function when cancel button is clicked
def on_cancel_click(cancel):
    clear_all()

# Section 8: Function for Resetting/Clearing Sections and Outputs

    - Function to reset dropdown selections and clear displayed messages after actions.

In [70]:
#clear all function
def clear_all():
    output_speciality.clear_output()
    speciality_selection.value = ''
    output_date.clear_output()
    date_selection.options = []
    date_selection.value = None
    output_time.clear_output()
    time_selection.options = []
    time_selection.value = None
    output_confirmation.clear_output()
    output_confirm.clear_output()

# Section 9: Observer - Selection and Clicks

    - Linking widgets to their corresponding handlers to make the interface interactive.

In [71]:
#observing selections
book_button.on_click(on_book_click)
speciality_selection.observe(on_pick_a_speciality, names = 'value')
date_selection.observe(on_pick_a_date, names='value')
time_selection.observe(on_pick_a_time, names='value')
confirm_button.on_click(on_confirm_click)
cancel_button.on_click(on_cancel_click)

# Section 10: Outputs - Print & Display

    - Welcome message!
    - Instructions to render the interactive interface in Jupyter.
### Challenges

    - Final on challenges: I had a struggle with typos, forgetting to add parantheses, which was causing a lot of errors.

In [72]:
 #displaying outputs
print('Welcome to SRH Hospital Appointment Booking System')
print('')
print('Please enter your patient number and click "Book"')
display(input_section, output_speciality, output_date, output_time, output_confirmation, output_confirm)

Welcome to SRH Hospital Appointment Booking System

Please enter your patient number and click "Book"


Output()

Output()

Output()

Output()

Output()